## Neural Network Details

This neural network takes as an input the raw board representation s of the position and its history, and outputs both move probabilities and a value, (p, v) =  f(s). 

The vector of move probabilities p represents the probability of selecting each move a, pa =  Pr(a| s). 
The value v is a scalar evaluation, estimating the probability of the current player winning from position s

### Input
The input to the neural network is a 19 ×  19 ×  17 image stack comprising 17 binary feature planes. Eight feature planes, Xt, consist of binary values indicating the presence of the current player’s stones ($X_t^i = 1$ if intersection i contains a stone of the player’s colour at time-step t; 0 if the intersection is empty, contains an opponent stone, or if t <  0). 

A further 8 feature planes, Yt, represent the corresponding features for the opponent’s stones. 

The final feature plane, C, represents the colour to play, and has a constant value of either 1 if black is to play or 0 if white is to play. 

These planes are concatenated together to give input features st =  [Xt, Yt, Xt−1, Yt−1,..., Xt−7, Yt−7, C]. 

History features Xt, Yt are necessary, because Go is not fully observable solely from the current stones, as repetitions are forbidden; similarly, the colour feature C is necessary, because the komi is not observable.

```
Input:  3 x 3 x 3 x 3 x 3

- a 3 x 3 grid of 3 x 3 spaces, where 1 indicates that x owns that space
- a 3 x 3 grid of 3 x 3 spaces, where 1 indicates that o owns that space
- a 3 x 3 grid of 3 x 3 spaces, where 0 indicates that x is playing, 1 indicates o is playing

Tic Tac Toe doesn't need history information
```

### Processing
The input features are processed by a residual tower that consists of a single convolutional block followed by either 19 or 39 residual blocks

The convolutional block applies the following modules:
* A convolution of 256 filters of kernel size 3 ×  3 with stride 1
* Batch normalization
* A rectifier nonlinearity

Each residual block applies the following modules sequentially to its input:
* A convolution of 256 filters of kernel size 3 ×  3 with stride 1
* Batch normalization
* A rectifier nonlinearity
* A convolution of 256 filters of kernel size 3 ×  3 with stride 1
* Batch normalization
* A skip connection that adds the input to the block
* A rectifier nonlinearity

The output of the residual tower is passed into two separate ‘heads’ for  computing the policy and value. 

The policy head applies the following modules:
* A convolution of 2 filters of kernel size 1 ×  1 with stride 1
* Batch normalization
* A rectifier nonlinearity
* A fully connected linear layer that outputs a vector of size 192 +  1  =   362, corresponding to logit probabilities for all intersections and the pass move

The value head applies the following modules:
* A convolution of 1 filter of kernel size 1 ×  1 with stride 1
* Batch normalization 
* A rectifier nonlinearity
* A fully connected linear layer to a hidden layer of size 256
* A rectifier nonlinearity
* A fully connected linear layer to a scalar
* A tanh nonlinearity outputting a scalar in the range [− 1,  1]

The overall network depth, in the 20- or 40-block network, is 39 or 79 parameterized layers, respectively, for the residual tower, plus an additional 2 layers for the policy head and 3 layers for the value head.

## Monte Carlo Tree Search Details

Each edge (s, a) in the search tree stores a prior probability  P(s, a), a visit count N(s, a), and an action value Q(s, a). 

Each simulation starts from the root state and iteratively selects moves that maximize an upper confidence bound 

Q(s, a) + U(s, a), where U(s, a) ∝  P(s, a) /  (1 +  N(s, a))

until a leaf node s′ is encountered. This leaf position is expanded and evaluated only once by the network to generate both prior probabilities and evaluation

f(s') => P(s',  \*), V(s')

Each edge (s, a) traversed in the simulation is updated to increment its visit count N(s, a), and to update its action value to the mean evaluation over these simulations (MCTS backpropagation)

For all edges that led to the current edge:  
N(s, a) += 1  
Q(s, a) = Q(s, a) * (N(s, a) - 1) / N(s, a) + V(s') / N(s, a) -- Linear interpolation between prior and new

MCTS may be viewed as a self-play algorithm that, given neural network parameters θ and a root position s, computes a vector of search probabilities recommending moves to play, π =  αθ(s), proportional to the exponentiated visit count for each move, πa ∝  N(s, a)1/τ, where τ is a temperature parameter.

AlphaGo Zero uses a much simpler variant of the asynchronous policy and value MCTS algorithm (APV-MCTS) used in AlphaGo Fan and AlphaGo Lee. Each node s in the search tree contains edges (s, a) for all legal actions. Each edge stores a set of statistics, N(s, a) is the visit count, W(s, a) is the total action value, Q(s, a) is the mean action value and P(s, a) is the prior probability of selecting that edge. 

Multiple simulations are executed in parallel on separate search threads. The algorithm proceeds by iterating over three phases, and then selects a move to play.

Selection.  The first in-tree phase of each simulation begins at the root node of the search tree, s0, and finishes when the simulation reaches a leaf node sL at time-step L. At each of these time-steps, t < L, an action is selected according to the statistics in the search tree, using a variant of the PUCT algorithm, where cpuct is a constant determining the level of exploration; this search control strategy initially prefers actions with high prior probability and low visit count, but asympotically prefers actions with high action value.

SELECTED ACTION = argmax(Q(s,a) + U(s,a))  
U(s,a) = $c_{puct}P(s,a) * (sum(N(s,b)) ^ .5 / (1 + N(s,a))$ where b represents all actions from s

Expand and evaluate. The leaf node sL is added to a queue for neural net-work evaluation, (di(p), v) =  fθ(di(sL)), where di is a dihedral reflection or rotation selected uniformly at random from i in [1..8]. Positions in the queue are evaluated by the neural network using a mini-batch size of 8; the search thread is locked until evaluation completes. The leaf node is expanded and each edge (sL, a) is initialized to {N(sL, a) =  0,  W(sL, a) =  0,  Q(sL, a) =  0,  P(sL, a) =  pa}; the value v is then backed up.

Backup. The edge statistics are updated in a backward pass through each step t ≤  L. The visit counts are incremented, N(st, at) =  N(st, at) +  1, and the action  value is updated to the mean value. We use virtual loss to ensure each thread evaluates different nodes.

Play. At the end of the search AlphaGo Zero selects a move a to play  in the root position s0, proportional to its exponentiated visit count, , where τ is a temperature parameter that controls the level of exploration. The search tree is reused at subsequent time-steps: the child node corresponding to the played action becomes the new root node; the subtree below this child is retained along with all its statistics, while the remainder of the tree is discarded. AlphaGo Zero resigns if its root value and best child value are lower than a threshold value.

SELECTED ACTION = $N(s_0,a)^{1/\tau}/\Sigma_bN(s_0,b)^{1/\tau}$

## Symbols

* s - Neural network input:   Board position
* v - Neural network output:  Estimated probability of current player winning from board position s
* p - Neural network output:  Vector of move probabilities
* f - Neural network:  f(s) => v, p
* $\theta$ - Network weights
* $\pi$ - MCTS output:  Vector of move probabilities
* P(s, a) - MCTS edge data:  Prior probability (NN prediction) of choosing action a in state s
* N(s, a) - MCTS edge data:  Number of visits to edge (s, a)
* Q(s, a) - MCTS edge data:  Value estimate of edge (s, a)
* c - Regularization parameter
* $\tau$ - Temperature value

## Parameter Updates

* Predicted - (v, p)
* Target    - (z, $\pi$)
* Loss      - (z - v) ^ 2 - $\pi^Tlog$p + c||$\theta$||$^2$

## Overall Algorithm

* Initialize the 0th iteration of the NN weights randomly
* Maintain a replay buffer for data from self-play games
* Maintain the "best" iteration of the NN weights (control network)
* Repeat until convergence:
  * Conduct self-play games:
    * At each step, run MCTS using the control network to get ($s_t, \pi_t, z_t$) and put in replay buffer
  * Train policy network using replay buffer
  * After x iterations, compare performance of policy and control network.  If policy network wins > 55% of games, make it the new control network

## Domain Knowledge Provided to AlphaGo

(1) AlphaGo Zero is provided with perfect knowledge of the game rules. These are used during MCTS, to simulate the positions resulting from a sequence of moves, and to score any simulations that reach a terminal state. Games terminate when both players pass or after 19 ×   19 ×  2 =  722 moves. In addition, the player is provided with the set of legal moves in each position.

(2) AlphaGo Zero uses Tromp–Taylor scoring during MCTS simulations and self-play training. This is because human scores (Chinese, Japanese or Korean rules) are not well-defined if the game terminates before territorial boundaries are resolved. However, all tournament and evaluation games were scored using Chinese rules.

(3) The input features describing the position are structured as a 19 ×  19  image; that is, the neural network architecture is matched to the grid-structure of the board.

(4) The rules of Go are invariant under rotation and reflection; this knowledge has been used in AlphaGo Zero both by augmenting the dataset during training to include rotations and reflections of each position, and to sample random rotations or reflections of the position during MCTS (see Search algorithm). Aside from komi, the rules of Go are also invariant to colour transposition; this knowledge is exploited by representing the board from the perspective of the current player (see Neural network architecture).

AlphaGo Zero does not use any form of domain knowledge beyond the points listed above. It only uses its deep neural network to evaluate leaf nodes and to select moves (see ‘Search algorithm’). It does not use any rollout policy or tree policy, and the MCTS is not augmented by any other heuristics or domain-specific rules. No legal moves are excluded—even those filling in the player’s own eyes (a standard heuristic used in all previous programs).

## Optimization Details

The batch-size is 32 per worker, for a total mini-batch size of 2,048. 

Each mini-batch of data is sampled uniformly at random from all positions of the most recent 500,000 games of self-play. 

Neural network parameters are optimized by stochastic gradient descent with momentum and learning rate annealing, using the loss in equation (1). 

The learning rate is annealed according to minibatch number:
* less than 400k:  10e-2 
* 400k to 600k:  10e-3
* greater than 600k: 10e-4

The momentum parameter is set to 0.9. 

The cross-entropy and MSE losses are weighted equally (this is reasonable because rewards are unit scaled,  r ∈  {− 1,  + 1}) and the L2 regularization parameter is set to c =  10−4. 

The optimiza-tion process produces a new checkpoint every 1,000 training steps. This checkpoint is evaluated by the evaluator and it may be used for generating the next batch of self-play games, as we explain next.

## Evaluation Details

* 400 games  
* 1600 MCTS simulations per move  
* Temperature epsilon small (argmax)  
* $\theta_* vs. \theta_{current}$ (Best network thus far vs. most recent checkpoint)  

If new checkpoint wins more than 55% of games, make it the new control network

## Self-Play Details

* 25000 games
* 1600 MCTS simulations per move
* First 30 moves:  Temperature = 1
* Subsequent moves:  Temperature = epsilon small

Additional exploration is achieved by adding Dirichlet noise to the prior probabilities in the root node s0, specifically P(s, a) =   (1 −   ε)pa +   εηa, where η ∼   Dir(0.03)  and  ε =  0.25; this noise ensures that all moves may be tried, but the search may still overrule bad moves. 

A game terminates at step T when both players pass, when the search value drops below a resignation threshold or when the game exceeds a maximum length; the game is then scored to give a final reward of r ∈ {− 1,+ 1} 

The data for each time-step t is stored as (st,πt,zt), where zt = ± r is the game winner from the perspective of the current player at step t